In [1]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression

In [2]:
in_data = []
with open('in.dta','r') as f:
    for line in f:
        in_data.append([float(i) for i in line.split()])
in_data = np.asarray(in_data)

In [3]:
out_data = []
with open('out.dta','r') as f:
    for line in f:
        out_data.append([float(i) for i in line.split()])
out_data = np.asarray(out_data)

In [4]:
#print(out_data)

In [12]:
def nonlinear_transform(X, k):
    new_X = []
    for x in X:
        x1 = x[0]
        x2 = x[1]
        new_x = [1, x1, x2, x1**2, x2**2, x1*x2, np.abs(x1-x2), np.abs(x1+x2)]
        new_x = [new_x[i] for i in range(k+1)]
        new_X.append(new_x)
    return np.asarray(new_X)

In [6]:
def linear_regression(X, y):
    X_plus = np.linalg.inv(X.transpose().dot(X)).dot(X.transpose())
    w = X_plus.dot(y)
    return(w)
    

In [22]:
def linear_regression_weight_decay(X, y, l):
    X_plus = np.linalg.inv(X.transpose().dot(X) + l*np.eye(X.shape[1])).dot(X.transpose())
    w = X_plus.dot(y)
    return(w)

In [8]:
def get_error(X, w, y):
    err = 0
    count = 0
    for x in X:
        err += (w.dot(x) - y[count])**2
        count += 1
    err = err/float(count)
    return err

In [9]:
def get_classification_error(X, w, y):
    correct_idx = []
    count = 0
    for x in X:
        if np.sign(w.dot(x)) == y[count]:
            correct_idx.append(count)
        count += 1
    class_err = 1-len(correct_idx)/float(count) 
    return class_err

In [10]:
def plot_target_in_x_space(x,y):
    neg_x = [x[i,0] for i in range(len(x[:,0])) if y[i] < 0]
    pos_x = [x[i,0] for i in range(len(x[:,0])) if y[i] > 0]
    neg_y = [x[i,1] for i in range(len(x[:,1])) if y[i] < 0]
    pos_y = [x[i,1] for i in range(len(x[:,1])) if y[i] > 0]
    plt.plot(neg_x, neg_y,'r*')
    plt.plot(pos_x, pos_y,'b*')
    plt.title('Target data')
    plt.show()

In [11]:
def plot_disagreement_in_x_space(X,y,Z,w):
    correct_idx = []
    count = 0
    for x in Z:
        if np.sign(w.dot(x)) == y[count]:
            correct_idx.append(count)
        count += 1
    neg_x = [X[i,0] for i in range(len(X[:,0])) if i not in correct_idx]
    pos_x = [X[i,0] for i in range(len(X[:,0])) if i in correct_idx]
    neg_y = [X[i,1] for i in range(len(X[:,1])) if i not in correct_idx]
    pos_y = [X[i,1] for i in range(len(X[:,1])) if i in correct_idx]
    plt.plot(neg_x, neg_y,'g*', label = 'correct')
    plt.plot(pos_x, pos_y,'y*', label = 'incorrect')
    plt.title('Linear Regression misclassified points')
    plt.xlabel('x_1')
    plt.ylabel('x_2')
    plt.legend()
    plt.savefig('Linear_Regression_disagreement.jpg')
    plt.show()

In [13]:
#using my own implementation, no weight decay, with validation
train_data = in_data[:25,:]
val_data = in_data[25:,:]
X = train_data[:,:2]
y = train_data[:,2]
val_X = val_data[:,:2]
val_y = val_data[:,2]
test_X = out_data[:,:2]
test_y = out_data[:,2]
#plot_target_in_x_space(X,y)
for k in [3,4,5,6,7]:
    print(k)
    
    Z = nonlinear_transform(X,k)
    w = linear_regression(Z,y)
    in_sample_err = get_classification_error(Z,w,y)
    print(in_sample_err)
    
    val_Z = nonlinear_transform(val_X,k)
    val_err = get_classification_error(val_Z,w,val_y)
    print(val_err)
    
    test_Z = nonlinear_transform(test_X,k)
    test_err = get_classification_error(test_Z,w,test_y)
    print(test_err)

3
0.44
0.3
0.42
4
0.32
0.5
0.416
5
0.08
0.2
0.188
6
0.04
0.0
0.084
7
0.04
0.1
0.072


In [21]:
#using my own implementation, no weight decay, swapping train and validation from before
train_data = in_data[25:,:]
# print(train_data.shape)
val_data = in_data[:25,:]
# print(val_data.shape)
X = train_data[:,:2]
y = train_data[:,2]
val_X = val_data[:,:2]
val_y = val_data[:,2]
test_X = out_data[:,:2]
test_y = out_data[:,2]
#plot_target_in_x_space(X,y)
for k in [3,4,5,6,7]:
    print(k)
    
    Z = nonlinear_transform(X,k)
    #print(Z.shape)
    w = linear_regression(Z,y)
    in_sample_err = get_classification_error(Z,w,y)
    print(in_sample_err)
    
    val_Z = nonlinear_transform(val_X,k)
    val_err = get_classification_error(val_Z,w,val_y)
    print(val_err)
    
    test_Z = nonlinear_transform(test_X,k)
    test_err = get_classification_error(test_Z,w,test_y)
    print(test_err)

3
0.4
0.28
0.396
4
0.3
0.36
0.388
5
0.2
0.2
0.284
6
0.0
0.08
0.192
7
0.0
0.12
0.196
